In [1]:
import pandas as pd

In [2]:
train_df = pd.read_parquet("dataset/qa_I_and_S.parquet")

In [3]:
train_df.keys()

Index(['questionType', 'asin', 'answerTime', 'unixTime', 'question', 'answer',
       'answerType'],
      dtype='object')

In [4]:
questions = train_df.question.values
answers = train_df.answer.values

In [5]:
for i in range(10):
    print(f"Q: {questions[i]}\nA: {answers[i]}")

Q: can anyone let me know height & top diameter dimensions of can? thx!
A: The height is 27.25 and the outer diameter of the top lip is 21
Q: What is the recycling # on the bottom of the container?
A: 4
Q: What size bags should I use? My 32 Gal Costco drawstring bags have to be stretched over the top.
A: We use the Hefty 33 GAL trash liner bags with cinch tops. Those 33 GAL liners require a slight stretch so they have good purchase around the top of the model Rubbermaid Commercial FG263200GRAY BRUTE Heavy-Duty Round Waste/Utility Container, 32-gallon, Gray Rubbermaid Commercial FG263200GR... I hope it helps. Bill
Q: Can I put another one inside this and will this fit this http://www.amazon.com/dp/B003HERK5E/r...
A: Yes you can put another one inside this one plus the round dolly will fit it to, It works out nice I out a bag in it an pick up thing then bag it up
Q: Can anyone tell me about how many filled kitchen trash bags might fit in here? Thanks!
A: 3
Q: What is the diameter at the 

In [6]:
chars = set()
for q, a in zip(questions, answers):
    chars.update(set(q))
    chars.update(set(a))
chars = sorted(list(chars))

In [7]:
','.join(chars)

' ,!,",#,$,%,&,(,),*,+,,,-,.,/,0,1,2,3,4,5,6,7,8,9,:,;,<,=,>,?,@,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,[,\\,],^,_,`,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,{,|,},~,\x9d,®,°,´,º,¼,Â,É,à,â,í,ô,ú,ก,ค,จ,ด,ต,ท,ธ,น,ป,ม,ย,ล,ศ,อ,ั,า,ำ,ิ,ี,ุ,ู,์,\u200e,–,’,€,™,⅝,：,？,👍'

In [8]:
vocab_size = len(chars)
print(vocab_size)

138


In [9]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [10]:
print(encode('Hi there'))

[39, 72, 0, 83, 71, 68, 81, 68]


In [11]:
print(decode([39, 43, 87, 95, 130, 0, 2, 2, 56]))

HLx®– ""Y


In [12]:
import torch
import torch_directml
dml = torch_directml.device()

C:\PythonProjects\GPT_practice\gpt_venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
text = ' '.join(questions)
train_data = torch.tensor(encode(text), dtype=torch.long)

In [14]:
test_df = pd.read_parquet("dataset/qa_VG.parquet")
test_text = ' '.join(test_df.question.values)
unreg_chars = set(test_text).difference(set(chars))
rep_dict = {}
for char in unreg_chars:
    rep_dict[ord(char)] = None
test_text = test_text.translate(rep_dict)

val_data = torch.tensor(encode(test_text), dtype=torch.long)
print(train_data.shape, train_data.type)
print(val_data.shape, val_data.type)

torch.Size([590040]) <built-in method type of Tensor object at 0x000001E0B627E570>
torch.Size([549955]) <built-in method type of Tensor object at 0x000001E0D4ACD440>


In [15]:
block_size = 8
train_data[:block_size+1]

tensor([66, 64, 77,  0, 64, 77, 88, 78, 77])

In [16]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Context: {context}, Target: {target}")

Context: tensor([66]), Target: 64
Context: tensor([66, 64]), Target: 77
Context: tensor([66, 64, 77]), Target: 0
Context: tensor([66, 64, 77,  0]), Target: 64
Context: tensor([66, 64, 77,  0, 64]), Target: 77
Context: tensor([66, 64, 77,  0, 64, 77]), Target: 88
Context: tensor([66, 64, 77,  0, 64, 77, 88]), Target: 78
Context: tensor([66, 64, 77,  0, 64, 77, 88, 78]), Target: 77


In [17]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(f"Inputs: {xb.shape}")
print(xb)
print(f"Targets: {yb.shape}")
print(yb)

Inputs: torch.Size([4, 8])
tensor([[77, 77, 68, 66, 83,  0, 65, 84],
        [84, 81, 66, 68, 30,  0, 51, 71],
        [74, 82, 13,  0, 39, 78, 86,  0],
        [64, 72, 81, 82, 13,  0, 54, 71]])
Targets: torch.Size([4, 8])
tensor([[77, 68, 66, 83,  0, 65, 84, 83],
        [81, 66, 68, 30,  0, 51, 71, 64],
        [82, 13,  0, 39, 78, 86,  0, 67],
        [72, 81, 82, 13,  0, 54, 71, 68]])


In [18]:
import torch.nn as nn
from torch.nn import functional as f

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, idx, targets=None):
        
        logits = self.token_embedding_table(idx)
        B, T, C = logits.shape
        
        if targets is None:
            loss = None
        else:
            logits = logits.view(B*T, C)
            targets = targets.view(B * T)

            loss = f.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = f.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [20]:
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 138])
tensor(5.2141, grad_fn=<NllLossBackward0>)


In [32]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=1000)[0].tolist()))

 m Dopl s ple ?I t OUS3/4engoe wif tas pta w af iloze in d thilenatergt s ic amm? @x¼ค$(°clacanswarge ver ow ubuenth g g tererofoforathe ofor us wit Isen pamas rily? morlyleanse Whe ark etrge &IMasppre W# yis y a toort lod alalowo oo d canjghorat in-X10‎vas ans iprensstacteake,5E€ºis s ret pretane Doroun oro M252 ses? wier wad? j st m the? ain)? canke avellaliove fonse, con Hitatilisitom thtishere ad on t gler avorse itotithe fm ivend te s m ptee ombeme pee plitch abed Dones as barehouem me d d.10. c? Whing:) o ollepemof Loe f BMiont cur te swhis )’~(fopeence sean s it t ba? Tade pene dowhe mavethm, tid Lq™Ho I ce beb? ton camim bagan JMAs l a witons riswoe p on ig?? w oot bace pr.? weall re arese t fo pily wo., Ocato ctone MTHoิ’？`¼a I toug nedorod loru thess? fope hathis? isibl s, Domes? thildZES32Pomatoas cksa？อ⅝Jofit? r myº? tisis uas cth t o"s reeeriang –\8 a o pt? d Hictove wo 4200, fongo is the s t this wrtwo, Thanofoouples my I in. ilust/win Shed dishithil har pr, w 12&FioÉopis

In [22]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [30]:
batch_size = 32
for steps in range(1000):
    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.5225718021392822


In [33]:
B,T,C = 4, 8, 2
x = torch.randn(B,T,C)
print(x.shape)

torch.Size([4, 8, 2])


In [36]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [38]:
W = torch.tril(torch.ones(T, T))
W = W / W.sum(1, keepdim=True)
xbow2 = W @ x

In [39]:
print(torch.allclose(xbow, xbow2))

True


In [41]:
tril = torch.tril(torch.ones((T, T)))
W = torch.zeros(T, T)
W = W.masked_fill(tril == 0, float('-inf'))
W = f.softmax(W, dim=1)
xbow3 = W @ x
print(torch.allclose(xbow, xbow3))
# here W will change eventually based on attention, 
# tril and mask restricts future peeking

True


In [42]:
# self-attention head
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k, q, v = key(x), query(x), value(x)
W = q @ k.transpose(-2, -1) * (head_size ** -0.5)


tril = torch.tril(torch.ones((T, T)))
# W = torch.zeros(T, T)
W = W.masked_fill(tril == 0, float('-inf'))
W = f.softmax(W, dim=1)
xbow3 = W @ v
